# Streamlit Chatbot Application with Gemini 3 Pro

> **Created by [Build Fast with AI](https://www.buildfastwithai.com)**

This notebook shows you how to build an interactive chatbot application using Streamlit and Gemini 3 Pro.

## What you'll learn:
- Building Streamlit applications
- Creating chatbot interfaces
- Managing conversation history
- Implementing session state
- Customizing chat UI
- Deploying chatbots

## 1. Installation and Setup

In [ ]:
!pip install -q streamlit google-generativeai

In [ ]:
import os
import google.generativeai as genai
from IPython.display import Markdown, display

In [ ]:
# Configure API key
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY', 'your-api-key-here')

genai.configure(api_key=GOOGLE_API_KEY)

## 2. Basic Streamlit Chatbot Code

Let's create a simple chatbot application.

In [ ]:
# Save this as chatbot_basic.py
basic_chatbot_code = '''
import streamlit as st
import google.generativeai as genai
import os

# Configure page
st.set_page_config(
    page_title="Gemini Chatbot",
    page_icon="🤖",
    layout="centered"
)

# Title
st.title("🤖 Gemini 3 Pro Chatbot")
st.caption("Built with Streamlit and Gemini 3 Pro")

# Get API key from environment or sidebar
api_key = os.environ.get('GOOGLE_API_KEY')
if not api_key:
    api_key = st.sidebar.text_input("Enter Google API Key", type="password")

if api_key:
    genai.configure(api_key=api_key)
    
    # Initialize session state for messages
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    if "chat" not in st.session_state:
        model = genai.GenerativeModel('gemini-3-pro')
        st.session_state.chat = model.start_chat(history=[])
    
    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # Chat input
    if prompt := st.chat_input("What would you like to know?"):
        # Add user message to history
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Get bot response
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                response = st.session_state.chat.send_message(prompt)
                st.markdown(response.text)
        
        # Add assistant response to history
        st.session_state.messages.append({"role": "assistant", "content": response.text})
    
    # Sidebar controls
    with st.sidebar:
        st.header("Controls")
        if st.button("Clear Chat History"):
            st.session_state.messages = []
            model = genai.GenerativeModel('gemini-3-pro')
            st.session_state.chat = model.start_chat(history=[])
            st.rerun()
else:
    st.warning("Please enter your Google API key to start chatting.")
'''

# Write to file
with open('chatbot_basic.py', 'w') as f:
    f.write(basic_chatbot_code)

print("Basic chatbot code saved to chatbot_basic.py")
print("\nTo run: streamlit run chatbot_basic.py")

## 3. Advanced Chatbot with System Instructions

In [ ]:
# Save this as chatbot_advanced.py
advanced_chatbot_code = '''
import streamlit as st
import google.generativeai as genai
import os

# Page config
st.set_page_config(
    page_title="Advanced Gemini Chatbot",
    page_icon="🤖",
    layout="wide"
)

# Custom CSS
st.markdown("""
<style>
    .stChatMessage {
        padding: 1rem;
        border-radius: 0.5rem;
    }
    .main-header {
        font-size: 2.5rem;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 1rem;
    }
</style>
""", unsafe_allow_html=True)

# Header
st.markdown('<p class="main-header">🤖 Advanced Gemini Chatbot</p>', unsafe_allow_html=True)
st.caption("Customizable AI Assistant powered by Gemini 3 Pro")

# Sidebar configuration
with st.sidebar:
    st.header("⚙️ Configuration")
    
    # API Key
    api_key = os.environ.get('GOOGLE_API_KEY')
    if not api_key:
        api_key = st.text_input("Google API Key", type="password")
    
    # Bot personality
    st.subheader("Bot Personality")
    personality = st.selectbox(
        "Choose personality",
        ["Professional Assistant", "Friendly Helper", "Technical Expert", "Creative Writer", "Custom"]
    )
    
    if personality == "Custom":
        system_instruction = st.text_area(
            "Custom System Instruction",
            "You are a helpful assistant."
        )
    else:
        personality_prompts = {
            "Professional Assistant": "You are a professional assistant. Be concise, accurate, and helpful.",
            "Friendly Helper": "You are a friendly and enthusiastic helper. Be warm, encouraging, and supportive.",
            "Technical Expert": "You are a technical expert. Provide detailed, accurate technical information with code examples.",
            "Creative Writer": "You are a creative writer. Be imaginative, expressive, and artistic in your responses."
        }
        system_instruction = personality_prompts[personality]
    
    # Model parameters
    st.subheader("Model Parameters")
    temperature = st.slider("Temperature", 0.0, 1.0, 0.7, 0.1)
    max_tokens = st.slider("Max Output Tokens", 256, 2048, 1024, 128)
    
    # Controls
    st.subheader("Controls")
    if st.button("🗑️ Clear Chat", use_container_width=True):
        st.session_state.messages = []
        st.session_state.chat = None
        st.rerun()
    
    if st.button("📥 Export Chat", use_container_width=True):
        if "messages" in st.session_state and st.session_state.messages:
            chat_text = "\n\n".join([
                f"{msg['role'].upper()}: {msg['content']}"
                for msg in st.session_state.messages
            ])
            st.download_button(
                "Download Chat",
                chat_text,
                "chat_history.txt",
                "text/plain",
                use_container_width=True
            )

# Main chat interface
if api_key:
    genai.configure(api_key=api_key)
    
    # Initialize session state
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    # Initialize or update chat with current settings
    if "chat" not in st.session_state or "last_config" not in st.session_state or \
       st.session_state.last_config != (system_instruction, temperature, max_tokens):
        
        generation_config = {
            "temperature": temperature,
            "max_output_tokens": max_tokens,
        }
        
        model = genai.GenerativeModel(
            'gemini-3-pro',
            generation_config=generation_config,
            system_instruction=system_instruction
        )
        st.session_state.chat = model.start_chat(history=[])
        st.session_state.last_config = (system_instruction, temperature, max_tokens)
    
    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # Chat input
    if prompt := st.chat_input("Ask me anything..."):
        # Add user message
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Get and display assistant response
        with st.chat_message("assistant"):
            with st.spinner("✨ Thinking..."):
                try:
                    response = st.session_state.chat.send_message(prompt)
                    st.markdown(response.text)
                    st.session_state.messages.append({"role": "assistant", "content": response.text})
                except Exception as e:
                    error_msg = f"Error: {str(e)}"
                    st.error(error_msg)
                    st.session_state.messages.append({"role": "assistant", "content": error_msg})
    
    # Show message count in sidebar
    with st.sidebar:
        st.divider()
        st.caption(f"💬 Messages: {len(st.session_state.messages)}")
else:
    st.info("👈 Please enter your Google API key in the sidebar to start chatting.")
    
    st.markdown("""
    ### Features:
    - 🎭 Multiple bot personalities
    - ⚙️ Adjustable model parameters
    - 💾 Export chat history
    - 🎨 Custom UI styling
    - 🔄 Real-time configuration updates
    """)
'''

with open('chatbot_advanced.py', 'w') as f:
    f.write(advanced_chatbot_code)

print("Advanced chatbot code saved to chatbot_advanced.py")
print("\nTo run: streamlit run chatbot_advanced.py")

## 4. Chatbot with Memory Management

In [ ]:
# Example: Managing conversation memory in a notebook
class ChatbotWithMemory:
    """Chatbot that manages conversation history."""
    
    def __init__(self, max_history: int = 10):
        self.max_history = max_history
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.chat = self.model.start_chat(history=[])
        self.full_history = []
    
    def send_message(self, message: str) -> str:
        """Send a message and get response."""
        # Get response
        response = self.chat.send_message(message)
        
        # Store in full history
        self.full_history.append({
            "role": "user",
            "content": message
        })
        self.full_history.append({
            "role": "assistant",
            "content": response.text
        })
        
        # Trim history if needed
        if len(self.full_history) > self.max_history * 2:
            self.full_history = self.full_history[-(self.max_history * 2):]
        
        return response.text
    
    def get_history(self):
        """Get conversation history."""
        return self.full_history
    
    def clear_history(self):
        """Clear conversation history."""
        self.chat = self.model.start_chat(history=[])
        self.full_history = []

# Test the chatbot
chatbot = ChatbotWithMemory(max_history=5)

# Have a conversation
messages = [
    "Hello! What's your name?",
    "Can you explain what Python is?",
    "What did I just ask you about?"
]

for msg in messages:
    print(f"\n{'='*80}")
    print(f"User: {msg}")
    response = chatbot.send_message(msg)
    print(f"\nAssistant: {response}")

## 5. Running Streamlit Apps from Colab/Jupyter

### Method 1: Using pyngrok (Recommended for Colab)

In [ ]:
# Install pyngrok for tunneling
!pip install -q pyngrok

# Note: You'll need to sign up for a free ngrok account and get your auth token
# Then run this code:

ngrok_tunnel_code = '''
from pyngrok import ngrok
import subprocess
import time

# Set your ngrok auth token (get it from https://dashboard.ngrok.com/get-started/your-authtoken)
# ngrok.set_auth_token("YOUR_NGROK_AUTH_TOKEN")

# Start streamlit in background
streamlit_process = subprocess.Popen(
    ["streamlit", "run", "chatbot_basic.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for streamlit to start
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print(f"\n✅ Streamlit app is running at: {public_url}")
print(f"\nClick the link above to access your chatbot!")
'''

print("To run Streamlit app from Colab:")
print(ngrok_tunnel_code)

### Method 2: Local Development

For local Jupyter notebooks, simply run:

```bash
# In terminal
streamlit run chatbot_basic.py
```

The app will open automatically in your browser at `http://localhost:8501`

## 6. Chatbot with Streaming Responses

In [ ]:
# Save this as chatbot_streaming.py
streaming_chatbot_code = '''
import streamlit as st
import google.generativeai as genai
import os

st.set_page_config(
    page_title="Streaming Chatbot",
    page_icon="💬",
    layout="centered"
)

st.title("💬 Streaming Chatbot")
st.caption("Watch responses appear in real-time!")

# API key
api_key = os.environ.get('GOOGLE_API_KEY')
if not api_key:
    api_key = st.sidebar.text_input("Google API Key", type="password")

if api_key:
    genai.configure(api_key=api_key)
    
    # Initialize session state
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    if "model" not in st.session_state:
        st.session_state.model = genai.GenerativeModel('gemini-3-pro')
        st.session_state.chat = st.session_state.model.start_chat(history=[])
    
    # Display messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # Chat input
    if prompt := st.chat_input("Your message..."):
        # Display user message
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Stream assistant response
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            full_response = ""
            
            # Stream the response
            for chunk in st.session_state.chat.send_message(prompt, stream=True):
                full_response += chunk.text
                message_placeholder.markdown(full_response + "▌")
            
            message_placeholder.markdown(full_response)
        
        st.session_state.messages.append({"role": "assistant", "content": full_response})
    
    # Sidebar
    with st.sidebar:
        st.header("Controls")
        if st.button("Clear Chat"):
            st.session_state.messages = []
            st.session_state.chat = st.session_state.model.start_chat(history=[])
            st.rerun()
else:
    st.warning("Please enter your API key to start.")
'''

with open('chatbot_streaming.py', 'w') as f:
    f.write(streaming_chatbot_code)

print("Streaming chatbot saved to chatbot_streaming.py")

## 7. Multi-Language Chatbot

In [ ]:
# Chatbot that can converse in multiple languages
class MultiLanguageChatbot:
    """Chatbot with multi-language support."""
    
    def __init__(self, language: str = "English"):
        self.language = language
        self.system_instruction = f"You are a helpful assistant. Always respond in {language}."
        self.model = genai.GenerativeModel(
            'gemini-3-pro',
            system_instruction=self.system_instruction
        )
        self.chat = self.model.start_chat(history=[])
    
    def change_language(self, language: str):
        """Change the conversation language."""
        self.language = language
        self.system_instruction = f"You are a helpful assistant. Always respond in {language}."
        self.model = genai.GenerativeModel(
            'gemini-3-pro',
            system_instruction=self.system_instruction
        )
        self.chat = self.model.start_chat(history=[])
    
    def send_message(self, message: str) -> str:
        """Send message and get response."""
        response = self.chat.send_message(message)
        return response.text

# Test with different languages
languages = ["English", "Spanish", "French", "German"]
question = "What is artificial intelligence?"

for lang in languages:
    print(f"\n{'='*80}")
    print(f"Language: {lang}")
    print(f"{'='*80}")
    
    bot = MultiLanguageChatbot(language=lang)
    response = bot.send_message(question)
    print(f"\n{response}")

## 8. Chatbot with Context Injection

In [ ]:
# Chatbot that can answer questions about specific context/documents
class ContextualChatbot:
    """Chatbot with injected context."""
    
    def __init__(self, context: str = ""):
        self.context = context
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.chat = self.model.start_chat(history=[])
        
        # If context provided, send it first
        if context:
            self.chat.send_message(f"Here is some context information:\n\n{context}\n\nPlease use this information to answer my questions.")
    
    def ask(self, question: str) -> str:
        """Ask a question about the context."""
        response = self.chat.send_message(question)
        return response.text
    
    def update_context(self, new_context: str):
        """Update the context."""
        self.context = new_context
        self.chat.send_message(f"Here is updated context:\n\n{new_context}")

# Example usage
company_context = """
TechCorp is a software company founded in 2020.
- CEO: Jane Smith
- Employees: 150
- Main Product: CloudSync - A cloud storage solution
- Revenue 2023: $10M
- Headquarters: San Francisco, CA
"""

bot = ContextualChatbot(context=company_context)

questions = [
    "Who is the CEO of TechCorp?",
    "What is their main product?",
    "How much revenue did they make in 2023?",
    "Where is the company located?"
]

for q in questions:
    print(f"\nQ: {q}")
    answer = bot.ask(q)
    print(f"A: {answer}")
    print("-" * 80)

## 9. Chatbot Analytics Dashboard

In [ ]:
# Chatbot with analytics tracking
import time
from collections import Counter

class AnalyticsChatbot:
    """Chatbot that tracks usage analytics."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.chat = self.model.start_chat(history=[])
        self.analytics = {
            "total_messages": 0,
            "total_tokens": 0,
            "response_times": [],
            "message_lengths": [],
            "topics": []
        }
    
    def send_message(self, message: str) -> str:
        """Send message and track analytics."""
        start_time = time.time()
        
        # Get response
        response = self.chat.send_message(message)
        
        # Calculate metrics
        response_time = time.time() - start_time
        message_length = len(message.split())
        
        # Update analytics
        self.analytics["total_messages"] += 1
        self.analytics["response_times"].append(response_time)
        self.analytics["message_lengths"].append(message_length)
        
        return response.text
    
    def get_analytics(self):
        """Get analytics report."""
        avg_response_time = sum(self.analytics["response_times"]) / len(self.analytics["response_times"]) if self.analytics["response_times"] else 0
        avg_message_length = sum(self.analytics["message_lengths"]) / len(self.analytics["message_lengths"]) if self.analytics["message_lengths"] else 0
        
        return {
            "Total Messages": self.analytics["total_messages"],
            "Average Response Time": f"{avg_response_time:.2f}s",
            "Average Message Length": f"{avg_message_length:.1f} words",
            "Fastest Response": f"{min(self.analytics['response_times']):.2f}s" if self.analytics["response_times"] else "N/A",
            "Slowest Response": f"{max(self.analytics['response_times']):.2f}s" if self.analytics["response_times"] else "N/A"
        }

# Test analytics
analytics_bot = AnalyticsChatbot()

# Have a conversation
messages = [
    "What is Python?",
    "Tell me about machine learning",
    "How do neural networks work?",
    "Explain deep learning in simple terms"
]

for msg in messages:
    response = analytics_bot.send_message(msg)
    print(f"User: {msg}")
    print(f"Bot: {response[:100]}...\n")

# Display analytics
print("\n" + "="*80)
print("CHATBOT ANALYTICS")
print("="*80)
analytics = analytics_bot.get_analytics()
for key, value in analytics.items():
    print(f"{key}: {value}")

## 10. Deployment Guide

### Deploying to Streamlit Cloud

1. **Prepare your repository:**
   ```bash
   # Create requirements.txt
   echo "streamlit>=1.28.0" > requirements.txt
   echo "google-generativeai>=0.3.0" >> requirements.txt
   ```

2. **Push to GitHub:**
   ```bash
   git add .
   git commit -m "Add chatbot app"
   git push
   ```

3. **Deploy on Streamlit Cloud:**
   - Go to [share.streamlit.io](https://share.streamlit.io)
   - Connect your GitHub repository
   - Select your main Python file
   - Add `GOOGLE_API_KEY` in Secrets management
   - Click Deploy!

### Alternative: Deploy on Hugging Face Spaces

1. Create a new Space on Hugging Face
2. Select Streamlit as the SDK
3. Upload your files
4. Add API key in Settings → Repository secrets

### Best Practices:

- ✅ Always use environment variables for API keys
- ✅ Add rate limiting for production
- ✅ Implement user authentication if needed
- ✅ Monitor usage and costs
- ✅ Add error handling and logging
- ✅ Test thoroughly before deploying

## Next Steps

Explore more advanced applications:
- Document Q&A with RAG
- Image analysis apps
- Multimodal chatbots
- Voice-enabled interfaces

---

## Learn More

Build production-ready AI applications with the **[Gen AI Crash Course](https://www.buildfastwithai.com/genai-course)** by Build Fast with AI!

**Created by [Build Fast with AI](https://www.buildfastwithai.com)**